# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load (Remember to Change These)
file_to_load = "clean_city_data.csv"

# Read Purchasing File and store into Pandas data frame
clean_city_data = pd.read_csv(file_to_load)

#Display the summary data frame
clean_city_data.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bethel,41.37,-73.41,296.48,88,75,3.10,US,1591915860
1,1,Goderich,43.75,-81.72,293.71,62,0,7.15,CA,1591916401
2,2,Hilo,19.73,-155.09,301.15,61,75,5.10,US,1591915700
3,3,Topolobampo,25.60,-109.05,307.59,52,5,7.70,MX,1591916402
4,4,Provideniya,64.38,-173.30,286.79,51,51,2.13,RU,1591915966


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure Gmaps
gmaps.configure(g_key)

In [4]:
# Use the Lat and Lng as locations and Humidity as the weight.
locations = clean_city_data[["Lat", "Lng"]].astype(float)

In [5]:
# Initialize (Heat)map
fig = gmaps.figure()

gmap_layer = gmaps.heatmap_layer(
    locations, 
    weights=clean_city_data["Humidity"],
    dissipating=False,
    max_intensity=100,
    point_radius = 1
)
fig.add_layer(gmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
vacation_city = clean_city_data[(clean_city_data['Max Temp'] > 298) 
                          & (clean_city_data['Max Temp'] < 303) 
                          &(clean_city_data['Cloudiness'] < 5)
                         &(clean_city_data['Humidity'] <50)]
vacation_city

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
47,47,North Platte,41.12,-100.77,302.59,28,1,2.60,US,1591916208
146,146,Praia,14.92,-23.51,302.15,45,0,3.60,CV,1591916458
182,182,Guerrero Negro,27.98,-114.06,301.67,27,0,7.28,MX,1591915952
234,234,San Quintín,30.48,-115.95,298.10,42,0,5.57,MX,1591915988
275,275,Burriana,39.89,-0.09,298.71,40,1,0.89,ES,1591916462
277,277,Ukiah,39.15,-123.21,302.15,35,1,3.60,US,1591916490
285,285,Zhangaözen,43.30,52.80,300.12,17,0,6.76,KZ,1591916509
312,312,Zhob,31.34,69.45,298.74,19,0,1.72,PK,1591916427
356,356,Denia,38.84,0.11,298.71,40,2,4.02,ES,1591916532
390,390,Ozark,37.02,-93.21,302.04,41,0,1.17,US,1591916543


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Store into hotel_df
hotel_df= pd.DataFrame(vacation_city)

In [8]:
hotel_df['Hotel Name'] = "NaN"
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
47,47,North Platte,41.12,-100.77,302.59,28,1,2.60,US,1591916208,NaN
146,146,Praia,14.92,-23.51,302.15,45,0,3.60,CV,1591916458,NaN
182,182,Guerrero Negro,27.98,-114.06,301.67,27,0,7.28,MX,1591915952,NaN
234,234,San Quintín,30.48,-115.95,298.10,42,0,5.57,MX,1591915988,NaN
275,275,Burriana,39.89,-0.09,298.71,40,1,0.89,ES,1591916462,NaN


In [9]:
# Set parameters to search for hotels within 5000 meters
target_radius = 5000
target_type = "hotel"
target_search = "hotel"

In [10]:
# Hit the Google Places API for each city's coordinates

hotel_df.loc[:,"Coordinates"] = hotel_df.loc[:,"Lat"].astype(str) +"," + hotel_df.loc[:,"Lng"].astype(str)
hotel_list = []

for location in hotel_df["Coordinates"]:
    params = {
        "location": location,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    gmaps_json = requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    
    try:
        hotel_list.append(gmaps_json["results"][0]["name"])
        
    except:
        hotel_list.append("N/A")
        print(f"No hotels within 5km of {location}.")

hotel_df["Hotel Name"] = hotel_list
hotel_df.loc[:, ~hotel_df.columns.str.contains('^Unnamed')]

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Coordinates
47,North Platte,41.12,-100.77,302.59,28,1,2.60,US,1591916208,Blue Spruce Motel,"41.12,-100.77"
146,Praia,14.92,-23.51,302.15,45,0,3.60,CV,1591916458,Hotel VIP Praia,"14.92,-23.51"
182,Guerrero Negro,27.98,-114.06,301.67,27,0,7.28,MX,1591915952,Hotel Los Caracoles,"27.98,-114.06"
234,San Quintín,30.48,-115.95,298.10,42,0,5.57,MX,1591915988,Don Eddie's Landing,"30.48,-115.95"
275,Burriana,39.89,-0.09,298.71,40,1,0.89,ES,1591916462,Hotel Vila-real Palace 4,"39.89,-0.09"
277,Ukiah,39.15,-123.21,302.15,35,1,3.60,US,1591916490,Best Western Orchard Inn,"39.15,-123.21"
285,Zhangaözen,43.30,52.80,300.12,17,0,6.76,KZ,1591916509,Hotel BasTau,"43.3,52.8"
312,Zhob,31.34,69.45,298.74,19,0,1.72,PK,1591916427,Rohail hotel Zhob,"31.34,69.45"
356,Denia,38.84,0.11,298.71,40,2,4.02,ES,1591916532,Hotel Port Dénia,"38.84,0.11"
390,Ozark,37.02,-93.21,302.04,41,0,1.17,US,1591916543,Quality Inn,"37.02,-93.21"


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

new_fig = fig = gmaps.figure()

markers = gmaps.marker_layer(locations, hotel_info)
new_fig.add_layer(gmap_layer)
new_fig.add_layer(markers)

# Display figure
new_fig

Figure(layout=FigureLayout(height='420px'))